# Imports

In [1]:
from lomond import WebSocket
import requests
import json

import re
from unidecode import unidecode

import numpy as np
import time

import pickle
with open('remove.dat', 'rb') as f:
    remove =  pickle.load(f)
with open('negative.dat', 'rb') as f:
    negative =  pickle.load(f)
    
from google import google
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

import warnings
warnings.simplefilter('ignore')

# Initiale ES Index

In [2]:
# es.indices.delete(index='hqtrivia', ignore=[400, 404])

# PUT /hqtrivia/
# {
#   "settings": {
#     "index": {
#       "store": {
#         "type": "fs"
#       },
#       "number_of_shards": 1,
#       "number_of_replicas": 1
#     },
#     "analysis" : {
#       "analyzer" : {
#         "stemming" : {
#           "tokenizer" : "standard",
#           "filter" : ["standard", "lowercase", "hqstemmer"]
#                 }
#             },
#       "filter" : {
#         "hqstemmer" : {
#           "type" : "stemmer",
#           "name" : "light_english"
#                 }
#             }
#         }
#   }
# }

# PUT /hqtrivia/description/_mapping
# {
#   "description": {
#     "properties": {
#       "description": {
#         "type": "text",
#         "store": true,
#         "fielddata": true,
#         "term_vector": "with_positions_offsets_payloads"
#       }
#     }
#   }
# }

# Get Bearer Token

In [3]:
headers = {
            'Host':             'api-quiz.hype.space',
            'Content-Type':     'application/json',
            'Accept-Encoding':  'gzip, deflate',
            'User-Agent':       'HQ-iOS/89 CFNetwork/808.2.16 Darwin/16.3.0',
            'Connection':       'keep-alive',
            'x-hq-device':      'iPhone6,1',
            'Accept':           '*/*',
            'Accept-Language':  'en-us',
            'x-hq-client':      'iOS/1.3.6 b89',
            'x-hq-test-key':    ''
}

number = input('Enter number: ')
json = {
            'method': 'sms',
            'phone':  f'+1{number}'
        }

verificationid = requests.request('POST', 
                                  'https://api-quiz.hype.space/verifications', 
                                  headers=headers, json=json)
verificationid = verificationid.json()['verificationId']

sms_code = input('Enter SMS code sent: ')
json = {
    'code':  sms_code
}

bearer = requests.request('POST', f'https://api-quiz.hype.space/verifications/{verificationid}', 
                          headers=headers, json=json)
bearer = bearer.json()['auth']['authToken'] 

bearer

# Connect to Web Socket

In [101]:
def connectwebsocket(socketurl, bearer):
    headers = {"Authorization": f"Bearer {bearer}",
               "x-hq-client": "Android/1.3.0"}
    websocket = WebSocket(socketurl)

    for header, value in headers.items():
        websocket.add_header(str.encode(header), str.encode(value))

    for msg in websocket.connect(ping_rate=5):
        if msg.name == "text":
            message = msg.text
            message = re.sub(r"[\x00-\x1f\x7f-\x9f]", "", message)
            message = json.loads(message)

            if message['type'] == 'question':
                question = unidecode(message['question'])
                options = [unidecode(option["text"])  for option in message["answers"]]
                print(f"\nQuestion {message['questionNumber']} out of {message['questionCount']}")
                print(f"{question}\n{options}\n")
#                 try:
                ES=esearch(question, options)
#                 except:
#                     print("Run ElasticSearch and initalize the index first.")
                GS=gsearch(question, options)
                print("FS: ", ES+GS)
                idx = np.argmax(ES+GS)
                if idx == 0: print('Predicted Answer: A')
                elif idx == 1: print('Predicted Answer: B')
                elif idx == 2: print('Predicted Answer: C')

#             elif message["type"] == "interaction" and message["itemId"] == "chat":
#                 comment = lower(re.sub(r'\W+\(\)', '', message["metadata"]["message"]))
#                 if comment=='a' or comment=='b' or comment=='c':
#                     print(upper(comment))

            elif message["type"] == "questionSummary":
                answers = [answer["correct"] for answer in message["answerCounts"]]
                idx = answers.index(True)
                if idx == 0: print('Actual Answer: A')
                elif idx == 1: print('Actual Answer: B')
                elif idx == 2: print('Actual Answer: C')

# TF Scoring 

In [102]:
def gsearch(question, options):
    question = question.lower()
    questionWords = question.replace("?", "").split()
    filtered_words = [word for word in questionWords if word not in remove]
    
    answers = []
    for answer in options:
        answers.append(answer.lower())
    
    negation = False    
    if len(set(questionWords)&negative)>0:
        negation = True

    M1=method1(filtered_words, answers, negation)
    M2=method2(filtered_words, answers, negation)
    return M1+M2

def method1(question, answers, negation):
    
    questionStr = ""
    for word in question:
        questionStr += word + " "

    search_results = google.search(questionStr, 1)

    words = ""
    for result in search_results:
        words += result.name + "\n"
        words += result.description + "\n"

    words = words.lower().split()

    prediction = []
    for answer in answers:
        answerWords = answer.lower().split()

        count = 0
        for word in words:
            for answerWord in answerWords:
                if answerWord in word:
                    count += 1
        prediction.append(count)
    
    if np.count_nonzero(prediction)!=0:
        prediction = np.true_divide(prediction, np.amax(prediction))
        
    if negation:
        prediction=np.negative(prediction)
        
    print("GS1: ", prediction)
    return prediction

def method2(question, answers, negation):
    
    questionStr = ""
    for word in question:
        questionStr += ' "' + word + '"'

    prediction = []
    for answer in answers:
        current = answer

        search_results = google.search(current + questionStr, 1)

        words = ""
        for result in search_results:
            words += result.name + "\n"
            words += result.description + "\n"

        words = words.lower().split()

        answerWords = answer.lower().split()

        count = 0
        for word in words:
            for answerWord in answerWords:
                if answerWord in word:
                    count += 1
        prediction.append(count)
        
    if np.count_nonzero(prediction)!=0:
        prediction = np.true_divide(prediction, np.amax(prediction))
        
    if negation:
        prediction=np.negative(prediction)
        
    print("GS2: ", prediction)
    return prediction

# ES Query BM-25 Scoring 

In [103]:
def esearch(ques, opts):  
    query = ' '.join([x for x in ques.lower().split() if x not in remove])+' '+' '.join([x for x in ' '.join(opts).split() if x not in remove]) 
    
    scores = np.array([])
    pages = google.search(query, 1)
    es.delete_by_query(index="hqtrivia", doc_type="description", body={"query": {"match_all": {}}})
    for p,page in enumerate(pages):
        if page.link==None:page.link=''
        if 'hq' not in page.link.lower():
            es.index(index="hqtrivia", doc_type='description', id=p, body={"description": page.description})
    time.sleep(1)

    for opt in opts:
        score=0
        res = es.search(index="hqtrivia", doc_type="description", body={"query": 
                                                                        {"match_phrase": 
                                                                         {'description': 
                                                                          {"query": opt,
                                                                           "slop":  34 
                                                                          }}}})
        for page in res['hits']['hits']:
            score += page['_score']
        scores=np.append(scores, score)
        
    if np.count_nonzero(scores)!=0:
        scores = np.true_divide(scores, np.amax(scores))
        
    if len(set(query.split())&negative)>0:
        scores=np.negative(scores)
        
    print("ES: ", scores)
    return scores

# Test on Simulation

In [105]:
try:
    res = requests.get('http://localhost:8732/').json()
    socketurl = res['broadcast']['socketUrl'].replace('https', 'wss')
    connectwebsocket(socketurl, bearer)
except:
    print("You need to launch the simulator first!")

Actual Answer: A

Question 2 out of 12
Which of these is a slang term for a super-exciting event?
['Apartment inverter', 'Barn burner', 'House shrinker']

ES:  [0. 1. 0.]
GS1:  [0, 0, 0]
GS2:  [0.88888889 1.         0.66666667]
FS:  [0.88888889 2.         0.66666667]
Predicted Answer: B
Actual Answer: B

Question 3 out of 12
Which of these was a key battle in the US Civil War?
['Bunker Hill', 'Gettysburg', 'Lexington & Concord']

ES:  [1.         0.66229237 0.41098091]
GS1:  [0. 1. 0.]
GS2:  [1.         0.45238095 0.9047619 ]
FS:  [2.         2.11467332 1.31574282]
Predicted Answer: B
Actual Answer: B

Question 4 out of 12
Which of these is a palindrome?
['Race car', 'Repeater', 'Tall pallet']

ES:  [0.93180387 1.         0.15455691]
GS1:  [1. 0. 0.]
GS2:  [1.         0.77777778 0.        ]
FS:  [2.93180387 1.77777778 0.15455691]
Predicted Answer: A
Actual Answer: A

Question 5 out of 12
What Marvel character is known for shouting, "It's clobberin' time"?
['Hulk', 'Thanos', 'Thing']

E

# Live Play

In [ ]:
res = requests.get('https://api-quiz.hype.space/shows/now').json()
if res['active']==True:
    socketurl= res['broadcast']['socketUrl'].replace('https', 'wss')
    connectwebsocket(socketurl, bearer)
else:
    print("Show's not up, yet!")